In [ ]:
import numpy as np
import pandas as pd
import uproot
import awkward_pandas
import matplotlib.pyplot as plt
import os
import math
import subprocess
import time
import itertools

In [ ]:
def getGarfieldPath():
    """
    """
    filename = 'GARFIELDPATH'
    try:
        with open(filename, 'r') as file:
            garfieldPath = file.read().strip()
            if not os.path.exists(garfieldPath):
                print(f"Error: File 'setupGarfield.sh' not found at {garfieldPath}.")
                return
            return garfieldPath

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return

def setupSimulation():
    """
    """
    #Get local Garfield path
    garfieldPath = getGarfieldPath()

    #Check for build/
    if not os.path.exists("build"):
        os.makedirs("build")
    
    #Make executable
    makeBuild = (
        f'source {garfieldPath} && '
        f'cd build && '
        f'cmake .. && '
        f'make'
    )
    result = subprocess.run(
        makeBuild,
        shell=True,
        check=True,
        executable='/bin/bash',
        capture_output=True,
        text=True
    )

    #Check for run number file
    if not os.path.exists("runNo"):
        with open(runNo, "w") as file:
            file.write('1000')
            
    return

In [ ]:
def defaultParam():
    """
    """
    defaultParam = {
        'padLength': 10.,
        'pitch': 55.,
        'gridStandoff': 100.,
        'gridThickness': .5,
        'holeRadius': 16.,
        'cathodeHeight': 400.,
        'thicknessSiO2': 5.,
        'fieldRatio': 50.,
        'numFieldLine': 25,
        'transparencyLimit': .1,
        'numAvalanche': 1000,
        'avalancheLimit': 200,
        'gasCompAr': 80.,
        'gasCompCO2': 20.,
    }
    return defaultParam

In [ ]:
def readParam():
    """
    """
    filename = 'runControl'
    param = {}
    
    try:
        with open(filename, 'r') as file:
            for line in file:
                line = line.strip()  # Remove leading/trailing whitespace
                if line.startswith('/') or not line:  # Skip comments and empty lines
                    continue
                # Split the line at the '='
                parts = line.split('=', 1)
                if len(parts) == 2:
                    key, value = parts[0].strip(), parts[1].strip()
                    value = value.rstrip(';') # Remove trailing semicolon
                    param[key] = value
                else:
                    print(f"Skipping malformed line: {line}")
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return {}
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return {}

    return param

In [ ]:
def writeRunControl(param=None):
    """
    """
    filename = 'runControl'

    if param is None or not param:
        print(f"No parameters to write to {filename}.")
        return

    try:
        with open(filename, 'r') as file:
            oldLines = file.readlines()  # Read all lines of the file
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return

    newLines = []
    for line in oldLines:
        line = line.strip()
        if line.startswith('/') or not line:
            newLines.append(line + '\n')  # Keep comments and empty lines
            continue

        parts = line.split('=', 1)
        if len(parts) == 2:
            key = parts[0].strip()
            if key in param:
                newLines.append(f"{key} = {param[key]};\n")  # Update value
            else:
                newLines.append(line + '\n') #keep original line
        else:
            newLines.append(line + '\n')  # Keep non-parameter lines

    try:
        with open(filename, 'w') as file:
            file.writelines(newLines)
    except Exception as e:
        print(f"An error occurred while writing to {filename}: {e}")
    
    return

In [ ]:
def calcPotentials(param=None):
    """
    """

    if param is None or not param:
        print(f"Invalid parameters.")
        return None
        
    convertEField = 0.1 # 1 kV/cm = 0.1 V/micron
    
    # Calculate the voltage required to achieve amplification field
    gridDistance = float(param['gridStandoff']) - float(param['gridThickness'])/2. #micron

    gridVoltage = float(param['fieldRatio'])*convertEField*gridDistance

    # Calculate for drift field
    cathodeDistance = float(param['cathodeHeight']) - float(param['gridThickness'])/2. #micron
    cathodeVoltage = convertEField*cathodeDistance + gridVoltage

    potentials = {
        'cathodeVoltage': -cathodeVoltage,
        'gridVoltage': -gridVoltage
    }
    
    return potentials

In [ ]:
def writeSIF(param=None):
    """
    """
    filename = os.path.join('./Geometry', 'FIMS.sif')

    if param is None or not param:
        print(f"Invalid parameters.")
        return None
    
    try:
        with open(filename, 'r') as file:
            sifLines = file.readlines()  # Read all lines of the file

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return

    potentials = calcPotentials(param)

    writeCathode = -1
    writeGrid = -1

    # Find the cathod and grid naming lines
    for i, line in enumerate(sifLines):
        if 'Name = "Cathode"' in line:
            writeCathode = i+1
        if 'Name = "Grid"' in line:
            writeGrid = i+1

    if writeCathode == -1 or 'Potential =' not in sifLines[writeCathode]:
        print('Error with cathode.')
        return
    if writeGrid == -1 or 'Potential =' not in sifLines[writeGrid]:
        print('Error with grid.')
        return
    
    sifLines[writeCathode] = f'\tPotential = {potentials['cathodeVoltage']}\n'
    sifLines[writeGrid] = f'\tPotential = {potentials['gridVoltage']}\n'
        
    try:
        with open(filename, 'w') as file:
            file.writelines(sifLines)
    except Exception as e:
        print(f"An error occurred while writing to {filename}: {e}")
        return
        
    return

In [ ]:
def writeParam(param=None):
    """
    """
    if param is None or not param:
        print(f"No parameters to write.")
        return    

    writeRunControl(param)
    writeSIF(param)
    
    return

In [ ]:
def resetParam(verbose=True):
    """
    """
    defaultParams = defaultParam()
    writeParam(defaultParams)

    if verbose:
        print('Parameters have been reset.')
    
    return defaultParams

In [ ]:
def getRunNumber():
    """
    """
    filename = 'runNo'

    try:
        with open(filename, 'r') as file:
            content = file.read().strip()
            runNo = int(content)
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return
    except ValueError:
        print(f"Error: Invalid number format in '{filename}")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return
    
    return runNo

In [ ]:
def runGmsh():
    with open(os.path.join(os.getcwd(), 'log/logGmsh.txt'), 'w+') as gmshOutput:
        startTime = time.monotonic()
        #subprocess.run(['gmsh', os.path.join('./Geometry/', 'FIMS.txt'), '-3'], stdout=gmshOutput)
        subprocess.run(['gmsh', os.path.join('./Geometry/', 'FIMS.txt'),
                        '-order', '2', 
                        '-optimize_ho',
                        '-clextend', '1',
                        '-setnumber', 'Mesh.OptimizeNetgen', '1',
                        '-setnumber', 'Mesh.MeshSizeFromPoints', '1',
                        '-3'],
                        stdout=gmshOutput)
        endTime = time.monotonic()
        gmshOutput.write(f'\n\nGmsh run time: {endTime - startTime} s')
    return

def runElmer():
    originalCWD = os.getcwd()
    os.chdir('./Geometry')

    if not os.path.exists("elmerResults"):
        os.makedirs("elmerResults")
        
    try:
        with open(os.path.join(originalCWD, 'log/logElmerGrid.txt'), 'w+') as elmerOutput:
            startTime = time.monotonic()
            subprocess.run(['ElmerGrid', '14', '2', 'FIMS.msh', '-out', 'elmerResults', '-autoclean'], stdout=elmerOutput)
            endTime = time.monotonic()
            elmerOutput.write(f'\n\nElmerGrid run time: {endTime - startTime} s')
            
        with open(os.path.join(originalCWD, 'log/logElmerSolver.txt'), 'w+') as elmerOutput:
            startTime = time.monotonic()
            subprocess.run(['ElmerSolver', 'FIMS.sif'], stdout=elmerOutput)
            endTime = time.monotonic()
            elmerOutput.write(f'\n\nElmerSolver run time: {endTime - startTime} s')
    finally:
        os.chdir(originalCWD)
    return

def runGarfield():
    originalCWD = os.getcwd()
    garfieldPath = getGarfieldPath()
    os.chdir('./build/')
    try:
        with open(os.path.join(originalCWD, 'log/logGarfield.txt'), 'w+') as garfieldOutput:
            startTime = time.monotonic()
            setupAvalanche = (
                f'source {garfieldPath} && '
                f'make && '
                f'./runAvalanche'
            )
            subprocess.run(setupAvalanche, stdout=garfieldOutput, shell=True, check=True)
            endTime = time.monotonic()
            garfieldOutput.write(f'\n\nGarfield run time: {endTime - startTime} s')
    finally:
        os.chdir(originalCWD)
    return

def runSimulation(param=None):

    if param is None or not param:
        print(f'No parameters to run.')
        return   

    print(f'Running simulation - Run number: {getRunNumber()}')
    writeParam(param)
    runGmsh()
    runElmer()
    runGarfield()
    return

In [ ]:
def varyParam(parameter, minVal, maxVal, numSteps=10):
    """
    """
    
    resetParam()
    param = readParam()
    
    if parameter not in param:
        print(f"Parameter '{parameter}' not able to be varied.")
        return None, None

    runNo = getRunNumber()
    print(f"Varying '{parameter}'. Initial run number: {runNo}")

    variedParam = np.linspace(minVal, maxVal, numSteps)
    runNos = [runNo + inRun for inRun in range(len(variedParam))]

    for inParam in variedParam:
        param[parameter] = inParam

        runSimulation(param)
            
    print(f"Done varying '{parameter}'.")

    param = resetParam()

    return variedParam, runNos

In [ ]:
def scanParam(parameters, minValues, maxValues, numSteps):
    """
    """

    if not (len(parameters) == len(minValues) == len(maxValues) == len(numSteps)):
        raise ValueError('Inputs must have the same length')

    resetParam()
    param = readParam()

    for inParam in parameters:
        if inParam not in param:
            print(f"Parameter '{inParam}' not able to be varied.")
            return None
    
    #Find range for each parameter
    allRanges = []
    for i, _ in enumerate(parameters):
        paramRange = np.linspace(minValues[i], maxValues[i], numSteps[i])
        allRanges.append(paramRange)

    print(f'Beginning scan through {parameters}. {math.prod(numSteps)} runs. Initial run: {getRunNumber()}')

    for scanParams in itertools.product(*allRanges):

        #Reset parameters to defaults
        param = resetParam(verbose=False)
        #Apply current parameter combination
        for i, inParam in enumerate(parameters):
            param[inParam] = scanParams[i]

        runSimulation(param)

    print(f'Done scan through {parameters}. Final run number: {getRunNumber()-1}')
    param = resetParam()
        
    return

In [ ]:
def scanValues(parameterDefinitions):
    """
    """

    #Separate the elements of scanning parameters
    parameters = []
    minValues = []
    maxValues = []
    numSteps = []
    
    for inDefinition in parameterDefinitions:

        if not (len(inDefinition) == 4):
            raise ValueError("Inputs must have the form: ['parameter', min, max, step]")
        
        inParam, inMin, inMax, inStep = inDefinition
            
        parameters.append(inParam)
        minValues.append(inMin)
        maxValues.append(inMax)
        numSteps.append(inStep)

    scanParam(parameters, minValues, maxValues, numSteps)
        
    return

In [ ]:
setupSimulation()
startTime = time.monotonic()
runSimulation(resetParam())
endTime = time.monotonic()
print(f'Test run takes ~{endTime-startTime:.1f}s.')

In [ ]:
#scanParam(['holeRadius', 'gridStandoff', 'fieldRatio'],
#          [16, 100, 30],
#          [20, 120, 30],
#          [3, 2, 1])
scanValues([
    ['holeRadius', 16, 20, 3],
    ['gridStandoff', 100, 120, 2],
    ['fieldRatio', 30, 30, 1]
])